In [18]:
import os

import mysql.connector
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

password = os.getenv("MYSQL_PASSWORD")

# Conectar a MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database="empresa"
)

cursor = connection.cursor()

In [19]:
# Crear tablas
cursor.execute("SHOW TABLES")

tables = cursor.fetchall()
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])

Tablas en la base de datos:
departamentos
empleados


In [2]:
# Crear tablas
cursor.execute("""
CREATE TABLE IF NOT EXISTS departamentos (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(100) NOT NULL
);
""")

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS empleados (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(100) NOT NULL,
    puesto VARCHAR(100),
    salario DECIMAL(10, 2),
    departamento_id INT,
    FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
);
""")

# Crear procedimiento almacenado
cursor.execute("""
DELIMITER $$
CREATE PROCEDURE AumentarSalario(
    IN dep_id INT,
    IN porcentaje DECIMAL(5,2)
)
BEGIN
    UPDATE empleados
    SET salario = salario + (salario * (porcentaje / 100))
    WHERE departamento_id = dep_id;
END $$
DELIMITER ;
""")

# Ejecutar el procedimiento
cursor.execute("CALL AumentarSalario(2, 10);")

# Verificar resultados
cursor.execute("SELECT * FROM empleados;")
for row in cursor.fetchall():
    print(row)

In [14]:
# Cerrar la conexión
cursor.close()
connection.close()

InternalError: Unread result found

In [16]:
import os
from urllib.parse import quote

import pandas as pd
from sqlalchemy import create_engine

user = "root"
password = quote(os.getenv("MYSQL_PASSWORD"))
host = "localhost"
port = "3306"
database = "empresa"

# Crear la URL de conexión
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de conexión
engine = create_engine(connection_string)

# Consultar todas las tablas
query = "SHOW TABLES"
df_tables = pd.read_sql(query, engine)
df_tables

,Tables_in_empresa
0,departamentos
1,empleados


In [5]:
# Consultar todas las tablas
query = "SELECT * FROM departamentos"
deps_df = pd.read_sql(query, engine)
deps_df

,id,nombre
0,1,Recursos Humanos
1,2,IT
2,3,Ventas


In [13]:
# Lee el archivo SQL y ejecuta el contenido
with open('procedure.sql', 'r') as file:
    sql_script = file.read()

# Ejecuta el procedimiento almacenado
with engine.connect() as connection:
    connection.execute(sql_script)

print("Procedimiento almacenado creado con éxito.")

ObjectNotExecutableError: Not an executable object: 'DELIMITER $$\n\nCREATE PROCEDURE AumentarSalario(\n    IN dep_id INT,\n    IN porcentaje DECIMAL(5,2)\n)\nBEGIN\n    UPDATE empleados\n    SET salario = salario + (salario * (porcentaje / 100))\n    WHERE departamento_id = dep_id'

In [38]:
import os

import mysql.connector
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

password = os.getenv("MYSQL_PASSWORD")

# Conectar a MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database="empresa"
)

cursor = connection.cursor()

query = "CALL AumentarSalario(1, 20);"


cursor.execute(query)
# Cerrar la conexión
cursor.close()
connection.close()

In [44]:
query = "SELECT * FROM empleados;"
deps_df = pd.read_sql(query, engine)
deps_df

,id,nombre,puesto,salario,departamento_id
0,1,Jonathan,Analista de Datos,65000.0,1
1,2,Ana,Ingeniera de Software,90750.0,2
2,3,Carlos,Vendedor,50000.0,3
